In [1]:
import os
import pandas as pd 
import numpy as np
import itertools as it
from pandarallel import pandarallel
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

import random

from sklearn.linear_model import LinearRegression
pandarallel.initialize()

import warnings
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
# get_input_path = lambda fname: os.path.abspath("/home/people/18200264/local_data/input_data/"+ fname)
# get_output_path = lambda fname: os.path.abspath("/home/people/18200264/local_data/output/GuanLab/CPTAC/"+ fname)

get_input_path = lambda fname: os.path.normpath('../local_data/processed_data/'+ fname)
get_output_path = lambda fname: os.path.normpath('../local_data/results/CPTAC/'+ fname)

In [3]:
file_cptac_transcriptomics = get_input_path('CPTAC_Transcriptomics_processed.parquet')
file_cptac_proteomics = get_input_path('CPTAC_Proteomics_processed.parquet')
file_cptac_sample_info = get_input_path('CPTAC_sample_info.parquet')

In [4]:
cptac_sample = pd.read_parquet(file_cptac_sample_info)[['Study']]
print("Dimensions: ", cptac_sample.shape)
cptac_sample[:2]

Dimensions:  (2021, 1)


,Study
C3L-00004,ccRCC
C3L-00010,ccRCC


In [5]:
cptac_proteomics = pd.read_parquet(file_cptac_proteomics)
print(cptac_proteomics.shape)
cptac_proteomics[:2]

(14792, 1227)


,01OV007,01OV017,01OV018,01OV023,01OV026,01OV029,01OV030,01OV039,01OV041,01OV047,...,X20BR002,X20BR005,X20BR006,X20BR007,X20BR008,X21BR001,X21BR002,X21BR010,X22BR005,X22BR006
A1BG,0.335938,-0.230469,0.1875,1.4375,-0.28125,-0.753906,0.101562,-1.117188,-0.140625,0.539062,...,1.171875,-1.125,0.882812,-1.0,0.972656,-1.363281,-2.078125,0.457031,-0.207031,-0.199219
A1CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cptac_transcriptomics = pd.read_parquet(file_cptac_transcriptomics)
print(cptac_transcriptomics.shape)
cptac_transcriptomics[:2]

(12581, 1901)


,X05BR044,X06BR006,X06BR005,X06BR003,X05BR045,X05BR016,X05BR043,X05BR042,X05BR038,X05BR029,...,C3L-00908,C3L-00907,C3L-00902,C3L-00817,C3L-00814,C3L-00813,C3L-00812,C3L-00800,C3L-00799,C3N-01808
ABCC9,0.335605,0.437866,0.289582,0.369700,0.274862,0.248999,0.300973,0.273792,0.365365,0.244641,...,0.106512,0.092456,0.147244,0.165088,0.112657,0.140601,0.172404,0.138284,0.102863,0.189004
AGR3,0.060490,0.196469,0.279060,0.450851,0.028310,0.426830,0.044012,0.000000,0.456943,0.000000,...,0.002412,0.000000,0.010735,0.000000,0.000000,0.005646,0.000000,0.007103,0.000000,0.000000


#### Process transcripts and proteins

1. Handling Missing values:
   - Proteins: Exclude proteins with >40% missing values 
   - Transcripts: Fill missing values with 0s

2. Protein isoforms / Repeated samples: 
   - Aggregating by computing the mean                         
                                                                               
3. Consider only the transcripts with greater than 0 variance   
4. Regress the cancer type from proteomics data 

In [7]:
common_proteins = np.intersect1d(cptac_transcriptomics.index, cptac_proteomics.index)
common_samples = np.intersect1d(cptac_transcriptomics.columns, cptac_proteomics.columns)
print("Common proteins = " + str(len(common_proteins)) + " and common samples = " + str(len(common_samples)))
cptac_transcriptomics_subset = cptac_transcriptomics.reindex(common_samples, axis=1).T
cptac_proteomics_subset = cptac_proteomics.reindex(common_samples, axis=1).reindex(common_proteins).T

Common proteins = 11058 and common samples = 1157


In [8]:
cptac_transcriptomics_study = cptac_transcriptomics_subset.merge(cptac_sample, right_index=True,
                                                               left_index=True).reset_index().sort_values(['Study', 'index']).set_index('index')
cptac_transcriptomics_study[:2]

,ABCC9,AGR3,TFF1,ABCC11,ABCC8,LMX1B,ABCA12,TMPRSS4,CEACAM5,PRAME,...,CNOT2,PPWD1,DDX24,IWS1,PDCL,CWC22,ZNF143,ZNF317,APOH,Study
index,,,,,,,,,,,,,,,,,,,,,
TCGA-A2-A0CM,0.438057,0.013974,0.009735,0.366004,0.013974,0.358992,0.256479,0.113873,0.447113,0.079922,...,0.650427,0.531460,0.704803,0.645089,0.535142,0.586527,0.521010,0.556072,0.013974,BrCa2016
TCGA-A2-A0D2,0.405471,0.237575,0.230412,0.073659,0.155145,0.510119,0.209696,0.091062,0.055237,0.505577,...,0.575599,0.527847,0.663623,0.592113,0.574797,0.540097,0.478587,0.571109,0.028262,BrCa2016


In [9]:
cptac_proteomics_study = cptac_proteomics_subset.merge(cptac_sample, right_index=True,
                                                       left_index=True).reset_index().sort_values(['Study', 
                                                                                                   'index']).set_index('index')
cptac_proteomics_study[:2]

,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAT,AAGAB,AAK1,AAMP,...,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Study
index,,,,,,,,,,,,,,,,,,,,,
TCGA-A2-A0CM,-0.640625,7.234375,NaN,NaN,0.305664,-3.3125,NaN,-1.417969,-0.328125,-1.158203,...,1.058594,NaN,NaN,NaN,NaN,-0.683594,-0.330078,-0.344727,-0.015625,BrCa2016
TCGA-A2-A0D2,-0.281250,-0.703125,NaN,NaN,0.789062,-1.1875,NaN,-0.832031,-0.367188,-1.148438,...,2.484375,1.9375,NaN,NaN,-1.290992,0.567383,0.875000,-0.657227,0.289062,BrCa2016


In [10]:
sample_count_per_study = cptac_proteomics_subset.merge(cptac_sample, right_index=True, left_index=True)['Study'].value_counts()
sample_count_per_study.sort_values(ascending=False, inplace=True)
sample_count_per_study

Study
Pdac        140
BrCa2020    122
LUAD        110
ccRCC       110
HNSCC       109
LSCC        108
OvCa2016    105
GBM          99
UCEC         95
OvCa2020     82
BrCa2016     77
Name: count, dtype: int64

In [11]:
def perform_prediction(protein, x_train, y_train, x_test, y_test):    
    # check if predictor df is null - np.nan is an instance of float
    if(isinstance(x_train, float) | isinstance(x_test, float)):
        return(pd.Series(index=y_test.index, name=protein, dtype=np.float64))
    
    common_samples = np.intersect1d(x_train.dropna().index, y_train.dropna().index)
    x_train_subset = x_train.reindex(common_samples)
    y_train_subset = y_train.reindex(common_samples)
    
    common_samples = np.intersect1d(x_test.dropna().index, y_test.dropna().index)
    x_test_subset = x_test.reindex(common_samples)
    y_test_subset = y_test.reindex(common_samples)
        
    if(len(x_train_subset) < 2):
        return(pd.Series(index=y_test_subset.index, name=protein, dtype=np.float64))
    if(isinstance(x_train_subset, pd.Series)):
        x_train_subset = x_train_subset.values.reshape(-1,1)
        x_test_subset = x_test_subset.values.reshape(-1,1)

    predictor = LinearRegression()
    predictor.fit(x_train_subset, y_train_subset)
    
    y_test_pred = pd.Series(predictor.predict(x_test_subset), index=y_test_subset.index, name=protein)
                        
    return(y_test_pred)

### Scenarios: 

1. self-mRNA
2. self-mRNA + CORUM
3. self-mRNA + STRING 400
4. self-mRNA + STRING 800
5. self-mRNA + STRING physical interactions 400
6. self-mRNA + STRING physical interactions 800
7. self-mRNA + VAE
8. self-mRNA + VAE + CORUM 
9. self-mRNA + VAE + STRING 400
10. self-mRNA + VAE + STRING 800
11. self-mRNA + VAE + STRING physical interactions 400
12. self-mRNA + VAE + STRING physical interactions 800

In [12]:
def get_studywise_data(study):
    # ensuring the proteomics and transcriptomics data is available for at least 60% of the samples
    proteomics = cptac_proteomics_study[cptac_proteomics_study['Study'] == study].drop(columns=['Study'])
    proteomics.dropna(thresh=0.6*(len(proteomics)), axis=1, inplace=True)    
    transcriptomics = cptac_transcriptomics_study[cptac_transcriptomics_study['Study'] == study].drop(columns=['Study']) 
    # Filling the data with less than 60% null values with 0
    transcriptomics = transcriptomics.dropna(thresh=0.6*(len(transcriptomics)), axis=1).fillna(0)
    
    # Select only those proteins with both proteomic and transcriptomic data available for at least 60% samples
    common_proteins = np.intersect1d(transcriptomics.columns, proteomics.columns)
    transcriptomics = transcriptomics.reindex(common_proteins, axis=1)
    proteomics = proteomics.reindex(common_proteins, axis=1)
    
    return(transcriptomics, proteomics)
  
def run_parallelized_baseline_pipeline(study, feature_set, desired_filename): 
    X, y = get_studywise_data(study)
    print("Number of proteins considered = ", len(y.columns))
    
    all_folds_results = []
    kf = KFold(n_splits=5, random_state=None, shuffle=False)
    for fold, (train, test) in enumerate(kf.split(X)):
        X_train = X.iloc[train]
        X_test = X.iloc[test]
        y_train = y.reindex(X_train.index)
        y_test = y.reindex(X_test.index)
        all_folds_results.append(y.columns.to_series().apply(lambda p: \
                                      perform_prediction(p, X_train[p], y_train[p], X_test[p], y_test[p])))
    pd.concat(all_folds_results, axis=1).to_parquet(get_output_path(desired_filename))
    print("Completed " + study)

In [13]:
parquet_extension = ".parquet"
BASELINE = 'Baseline'
baselinefile_prefix = "baseline_LR_selfmRNA_"

In [14]:
print(BASELINE)
for index, value in sample_count_per_study.head(6).items():
    desired_filename = baselinefile_prefix + index + parquet_extension
    run_parallelized_baseline_pipeline(index, BASELINE, desired_filename)

Baseline
Number of proteins considered =  6433
Completed Pdac
Number of proteins considered =  7421
Completed BrCa2020
Number of proteins considered =  7310
Completed LUAD
Number of proteins considered =  6413
Completed ccRCC
Number of proteins considered =  7113
Completed HNSCC
Number of proteins considered =  7905
Completed LSCC


In [ ]:
file_cptac_proteomics = get_input_path('CPTAC_proteomics_linkedomics.parquet')